# <font color='#42AAFF'>Содержание</font><a id='toc0_'></a>  <a id='toc0_'></a>    
1. [<font color='#42AAFF'>Импорт данных</font>](#toc1_)    
2. [<font color='#42AAFF'>Подготовка данных для обучения классификатора</font>](#toc2_)    
3. [<font color='#42AAFF'>Оптимизация гиперпараметров классификатора</font>](#toc3_)    
4. [<font color='#42AAFF'>Обучение оптимизированной модели</font>](#toc4_)    
5. [<font color='#42AAFF'>Значения целевой метрики на тренировочной и валидационных выборках</font>](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=true
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [26]:
import pandas as pd
import numpy as np
import time
from catboost import Pool, cv, CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from joblib import dump, load

# 1. <a id='toc1_'></a>[<font color='#42AAFF'>Импорт данных</font>](#toc0_)

Импортируем признаки базовой, тренировочной и тестовой выборок, преобразуем их в массивы

In [2]:
features_base = pd.read_csv("features_base.csv", index_col=0).values
features_train = pd.read_csv("features_train.csv", index_col=0).values
target_train = pd.read_csv("target_train.csv", index_col=0).values

Импортируем индексы кандидатов выбранной конфигурации кластеров в терминах базовых индексов, расстояния между целевыми векторами и кандидатами и словарь соответствия строк базового набора и названия объекта:

In [3]:
idx = pd.read_csv("index_train.csv", index_col=0).values
vecs = pd.read_csv("vecs_train.csv", index_col=0).values
base_index = pd.read_csv("base_index_train.csv", index_col=0,header=None).to_dict()[1]

# 2. <a id='toc2_'></a>[<font color='#42AAFF'>Подготовка данных для обучения классификатора</font>](#toc0_)

Напишем функцию, позволяющую подготовить данные для обучения бинарного классификатора catboost. Матрица признаков состоит из 72 элементов целевого вектора, 72 элеменетов вектора-кандидата и квадрата расстояния между ними. Целевой признак в случае совпадения целевого вектора и вектора кандидата равен 1,  в противном случае 0.

In [37]:
def prepare_data(idx,dist,x_base,base_index,x,y):
    idx_flat = idx.flatten()
    dist = dist.flatten().reshape(idx.shape[0]*idx.shape[1],1)
    data1 = x_base[idx_flat]
    data2 = []
    for ind in range(idx.shape[0]):    
        data2.extend(np.repeat([x[ind]],idx.shape[1],axis=0))
    data2 = np.array(data2)  
    X = np.hstack((data2,data1,dist))

    Y = []
    for target, el in zip(y[:,0].tolist(), idx.tolist()):
        for r in el:
            Y.append(int(target in base_index[r]))
    Y = np.array(Y)  
    return X,Y

Воспользуемся функцией подготовки данных для генерации признаков тренировочной выборки для обучения catboost:

In [39]:
%%time
X_train, Y_train = prepare_data(idx,vecs,features_base,base_index,features_train,target_train)

Wall time: 21.3 s


Так как на каждый целевой вектор мы отобрали 50 кандидатов, то длина целевого признака равен 100тыс. * 50 = 5 млн, а размеры матрицы признаков -  5млн*145.

In [7]:
Y_train.shape

(5000000,)

# 3. <a id='toc3_'></a>[<font color='#42AAFF'>Оптимизация гиперпараметров классификатора</font>](#toc0_)

Выполним разбиение на тренировочную и валидационные выборки:

In [8]:
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=0, shuffle=False)

In [9]:
train_pool = Pool(x_train, y_train)
val_pool = Pool(x_val, y_val)

Зададим параметры оптимизации catboost:

In [11]:
grid = {    
    'learning_rate': [0.03, 0.06],
    'depth':[3,6],
    #'l2_leaf_reg': [2, 3, 4],
    'boosting_type': ['Plain']
}

Выполним оптимизацию:

In [12]:
%%time
grid_search_model = CatBoostClassifier(
    iterations=2000,
    random_seed=42,    
    auto_class_weights='Balanced',
    loss_function="Logloss")
grid_search_result = grid_search_model.grid_search(grid, train_pool, verbose=20)

0:	learn: 0.6767146	test: 0.6766879	best: 0.6766879 (0)	total: 534ms	remaining: 17m 47s
1:	learn: 0.6618084	test: 0.6617287	best: 0.6617287 (1)	total: 958ms	remaining: 15m 57s
2:	learn: 0.6477820	test: 0.6476264	best: 0.6476264 (2)	total: 1.34s	remaining: 14m 51s
3:	learn: 0.6350217	test: 0.6348407	best: 0.6348407 (3)	total: 1.75s	remaining: 14m 32s
4:	learn: 0.6231462	test: 0.6229796	best: 0.6229796 (4)	total: 2.14s	remaining: 14m 14s
5:	learn: 0.6120842	test: 0.6118592	best: 0.6118592 (5)	total: 2.56s	remaining: 14m 10s
6:	learn: 0.6012352	test: 0.6009766	best: 0.6009766 (6)	total: 2.98s	remaining: 14m 7s
7:	learn: 0.5923190	test: 0.5920332	best: 0.5920332 (7)	total: 3.38s	remaining: 14m 1s
8:	learn: 0.5825417	test: 0.5822156	best: 0.5822156 (8)	total: 3.78s	remaining: 13m 56s
9:	learn: 0.5743731	test: 0.5739559	best: 0.5739559 (9)	total: 4.17s	remaining: 13m 50s
10:	learn: 0.5666676	test: 0.5662395	best: 0.5662395 (10)	total: 4.57s	remaining: 13m 45s
11:	learn: 0.5589570	test: 0.558

# 4. <a id='toc4_'></a>[<font color='#42AAFF'>Обучение оптимизированной модели</font>](#toc0_)

Наилучший результат получили для модели со следующими параметрами:

In [13]:
grid_search_result['params']

{'depth': 6, 'learning_rate': 0.06, 'boosting_type': 'Plain'}

Обучим модель с наилучшими параметрами на тренировочной части и валидируем её на валидационной:

In [40]:
model = CatBoostClassifier(
    **grid_search_result['params'],
    iterations=4000,
    random_seed=42,    
    auto_class_weights='Balanced',
    loss_function="Logloss",
    early_stopping_rounds=50    
)
model.fit(train_pool, verbose=20, eval_set=val_pool)

0:	learn: 0.6576356	test: 0.6574679	best: 0.6574679 (0)	total: 741ms	remaining: 49m 22s
20:	learn: 0.4016372	test: 0.4001299	best: 0.4001299 (20)	total: 14.1s	remaining: 44m 34s
40:	learn: 0.3388247	test: 0.3371600	best: 0.3371600 (40)	total: 26.2s	remaining: 42m 7s
60:	learn: 0.3100510	test: 0.3081326	best: 0.3081326 (60)	total: 38.2s	remaining: 41m 6s
80:	learn: 0.2892435	test: 0.2872724	best: 0.2872724 (80)	total: 50.2s	remaining: 40m 31s
100:	learn: 0.2741837	test: 0.2725237	best: 0.2725237 (100)	total: 1m 2s	remaining: 40m 22s
120:	learn: 0.2628511	test: 0.2617753	best: 0.2617753 (120)	total: 1m 14s	remaining: 39m 55s
140:	learn: 0.2533880	test: 0.2527920	best: 0.2527920 (140)	total: 1m 26s	remaining: 39m 38s
160:	learn: 0.2436868	test: 0.2435470	best: 0.2435470 (160)	total: 1m 39s	remaining: 39m 29s
180:	learn: 0.2364821	test: 0.2368221	best: 0.2368221 (180)	total: 1m 51s	remaining: 39m 14s
200:	learn: 0.2317321	test: 0.2324494	best: 0.2324494 (200)	total: 2m 3s	remaining: 38m 58

In [41]:
dump(model,'model.joblib')

['model.joblib']

In [42]:
model1 = load('model.joblib')

# 5. <a id='toc5_'></a>[<font color='#42AAFF'>Значения целевой метрики на тренировочной и валидационных выборках</font>](#toc0_)

Подготовим функцию для расчёта метрики accuracy@5:

In [43]:
def calc_accuracy5(x,y,model,n_samples):
    proba = model.predict_proba(x)[:,1]
    proba = proba.reshape(int(x.shape[0]/n_samples),n_samples)
    y = np.array(y).reshape(int(x.shape[0]/n_samples),n_samples)
    acc5 = 0
    for val,prob in zip(y,proba):
        acc5 += np.sum(val[np.argsort(prob)[-5:]])
    acc5 = acc5/len(y)
    return acc5

In [44]:
calc_accuracy5(x_train,y_train,model1,50)

0.7464777777777778

In [45]:
calc_accuracy5(x_val,y_val,model1,50)

0.7315